In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

# Tutorial

A few examples to show how to use the classes of the package `catede` in order to estimate quantities such the Shannon entropy and the Kullback-Leibler divergence from data. 

## First `Experiment` class 
In these examples we generate $K=20^3$ categories distributed as sequeunces of length $L=3$ generated as a $20$ states Markov chain with random transition matrix.

In [2]:
from catede.handle_ngrams import markov_class
from catede.estimate import Experiment

seed_1 = 13273                                          # rng seed

#  simulation  #
A = 20                                                  # n. of states
L = 3                                                   # length of the L-grams
K = A ** L                                              # n. categories a priori

mobj_1 = markov_class( L, n_states=A, seed=seed_1 )     # random Markov matrix

size = int(5e4)                                         # sample size
seqs_1 = mobj_1.generate_counts( size, seed=seed_1 )    # generate histogram of counts
exact_sh_entropy_1 = mobj_1.exact_shannon()             # exact Shannon entropy

exp_1 = Experiment( seqs_1, categories=K )              # first experiment

## Shannon entropy estimation

$$ S = - \sum_{i=1}^{K} q_{i} \log q_{i}$$

In [3]:
shannon_1_naive = exp_1.entropy( method='naive' ) 
shannon_1_CAE = exp_1.entropy( method='CAE' ) 
shannon_1_NSB, shannon_1_NSBstd = exp_1.entropy( method='NSB', error=True, verbose=False ) 

print("Shannon entropy")
print( f"exact : { exact_sh_entropy_1:.3f}" )
print( f"naive : {shannon_1_naive:.3f}" )
print( f"CAE : {shannon_1_CAE:.3f}" )
print( f"NSB : {shannon_1_NSB:.3f}", r"+-", f"{shannon_1_NSBstd:.3f}" )

Shannon entropy
exact : 8.592
naive : 8.513
CAE : 8.574
NSB : 8.593 +- 0.003


## Simpson index estimation

$$ \lambda = \sum_{i=1}^{K} {q_{i}}^2

In [4]:
exact_si_idx_1 = mobj_1.exact_simpson()
simpson_1_naive = exp_1.simpson( method='naive' )
simpson_1_CAE = exp_1.simpson( method='CAE' ) 
simpson_1_NSB, simpson_1_NSBstd = exp_1.simpson( method='NSB', error=True, n_bins=100 )

print("Simpson index")
print( f"exact : {exact_si_idx_1:.6f}" )
print( f"naive : {simpson_1_naive:.6f}" )
print( f"CAE : {simpson_1_CAE:.6f}" )
print( f"NSB : {simpson_1_NSB:.6f}", r"+-", f"{simpson_1_NSBstd:.6f}" )

Simpson index
exact : 0.000227
naive : 0.000246
CAE : 0.000238
NSB : 0.000232 +- 0.000001


## `Divergence` class

In [5]:
from catede.estimate import Divergence

# simulation of an independent second system
seed_2 = 5119                                           # rng seed

mobj_2 = markov_class( L, n_states=A, seed=seed_2 )     # random Markov matrix generation
seqs_2 = mobj_2.generate_counts( size, seed=seed_2 )    # generate histogram of counts
exact_sh_entropy_2 = mobj_2.exact_shannon()             # exact Shannon entropy  
exp_2 = Experiment( seqs_2, categories=K )              # second experiment
div_to1from2 = Divergence( exp_1, exp_2 )               # divergence class

## Kullback-Leibler divergence estimation

$$ D_{\rm KL} = \sum_{i=1}^{K} q_{i} \log \frac{q_{i}}{t_{i}}$$

In [6]:
# Kullback Leibler divergence estimation #
exact_DKL_to1from2 = mobj_1.exact_kullbackleibler( mobj_2 )
kullback_naive = div_to1from2.kullback_leibler(method='naive')
kullback_Z = div_to1from2.kullback_leibler( method='Zhang-Grabchak', error=True ) 
kullback_CMW, kullback_CMWstd = div_to1from2.kullback_leibler( method='CMW', error=True ) 

print("Kullback Leilber divergence")
print( f"exact : { exact_DKL_to1from2:.3f}" )
print( f"naive : {kullback_naive:.3f}" )
print( f"Z : {kullback_Z:.3f}" )
print( f"CMW : {kullback_CMW:.3f}", r"+-", f"{kullback_CMWstd:.3f}" )

Kullback Leilber divergence
exact : 0.949
naive : 0.589
Z : 0.826
CMW : 0.935 +- 0.010


## Squared Hellinger divergence estimation

$$ D_{\rm H}^2 = 1 - \sum_{i=1}^{K} \sqrt{q_{i}} \sqrt{t_{i}} $$

In [7]:
# Hellinger divergence estimation #
exact_DH_to1from2 = mobj_1.exact_squared_hellinger( mobj_2 )
hellinger_naive = div_to1from2.squared_hellinger(method='naive')
hellinger_CMW, hellinger_CMWstd = div_to1from2.squared_hellinger( method='CMW', error=True, n_bins=1 ) 

print("Hellinger divergence")
print( f"exact : { exact_DH_to1from2:.3f}" )
print( f"naive : {hellinger_naive:.3f}" )
print( f"CMW : {hellinger_CMW:.3f}", r"+-", f"{hellinger_CMWstd:.3f}" )

Hellinger divergence
exact : 0.204
naive : 0.278
CMW : 0.201 +- 0.002
